# Makers

In this tutorial, you will:

- Learn about `Maker`s.
- Understand how to use a `Maker` to update the parameters of jobs in a flow.


## What is a `Maker`?

A `Maker` is class that makes it convenient to update parameters on-the-fly in a workflow. It is best explained by example.

Let's start by defining a simple `Maker` that either adds or multiplies two numbers together, which we will do twice to make a flow. Note that all classes inheriting from the `Maker` base class must have a `name` variable and a `make` method.


In [11]:
from dataclasses import dataclass
from jobflow import job, Flow, Maker
from jobflow.managers.local import run_locally

@dataclass
class AddMaker(Maker):
    name: str = "Add Maker"
    operation: str = "add"

    @job
    def make(self, a, b):
        if self.operation == "add":
            return a + b
        elif self.operation == "mult":
            return a * b
        else:
            raise ValueError(f"Unknown operation: {self.operation}")


job1 = AddMaker().make(a=2, b=3)
job2 = AddMaker().make(a=job1.output, b=4)

flow = Flow([job1, job2])
responses = run_locally(flow)

2023-06-07 23:09:07,830 INFO Started executing jobs locally
2023-06-07 23:09:07,834 INFO Starting job - Add Maker (65b79c8c-fd18-4cd2-9207-19c027ad2fa9)
2023-06-07 23:09:07,836 INFO Finished job - Add Maker (65b79c8c-fd18-4cd2-9207-19c027ad2fa9)
2023-06-07 23:09:07,837 INFO Starting job - Add Maker (e9f64f22-d12d-45ca-898e-81e59a0c300b)
2023-06-07 23:09:07,839 INFO Finished job - Add Maker (e9f64f22-d12d-45ca-898e-81e59a0c300b)
2023-06-07 23:09:07,839 INFO Finished executing jobs locally


In [12]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

65b79c8c-fd18-4cd2-9207-19c027ad2fa9 -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
e9f64f22-d12d-45ca-898e-81e59a0c300b -> {1: Response(output=9, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


Right now, nothing particularly special has happened here. But what if you had a much more complex workflow with many steps and you wanted to change the `AddMaker` keyword arguments, but only for a few individual jobs in the flow? That's where the `Maker` comes in handy. Let's see how it works.


In [13]:
from dataclasses import dataclass
from jobflow import job, Flow, Maker
from jobflow.managers.local import run_locally

@dataclass
class AddMaker(Maker):
    name: str = "Add Maker"
    operation: str = "add"

    @job
    def make(self, a, b):
        if self.operation == "add":
            return a + b
        elif self.operation == "mult":
            return a * b
        else:
            raise ValueError(f"Unknown operation: {self.operation}")

@dataclass
class SubtractMaker(Maker):
    name: str = "Subtract Maker"

    @job
    def make(self, a, b):
        return b - a

job1 = AddMaker().make(a=2, b=3)
job2 = SubtractMaker().make(a=job1.output, b=4)
flow = Flow([job1, job2])

In [14]:
flow.update_maker_kwargs({"operation": "mult"}, name_filter="Add Maker")
responses = run_locally(flow)

2023-06-07 23:09:08,080 INFO Started executing jobs locally
2023-06-07 23:09:08,081 INFO Starting job - Add Maker (81318e37-b1f5-4cd8-a189-46a6c6b427a1)
2023-06-07 23:09:08,083 INFO Finished job - Add Maker (81318e37-b1f5-4cd8-a189-46a6c6b427a1)
2023-06-07 23:09:08,084 INFO Starting job - Subtract Maker (384c2c77-2640-4187-8b37-9c1c2cab0384)
2023-06-07 23:09:08,087 INFO Finished job - Subtract Maker (384c2c77-2640-4187-8b37-9c1c2cab0384)
2023-06-07 23:09:08,089 INFO Finished executing jobs locally


In [15]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

81318e37-b1f5-4cd8-a189-46a6c6b427a1 -> {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
384c2c77-2640-4187-8b37-9c1c2cab0384 -> {1: Response(output=-2, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


In this example, we have updated the keyword arguments ("kwargs") of jobs in a flow using a `name_filter` and the `update_maker_kwargs` function, which functions because the classes in the flow are themselves `Maker` objects.